In [ ]:

import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve


In [ ]:

zip_path = "T_ONTIME_REPORTING_20250629_200724.zip"

with zipfile.ZipFile(zip_path) as z:
    csv_name = z.namelist()[0]
    df = pd.read_csv(z.open(csv_name), low_memory=False)

print("Original shape:", df.shape)
df.head()


In [ ]:

columns = [
    "ARR_DELAY",
    "DEP_DELAY",
    "CANCELLED",
    "DIVERTED",
    "MONTH",
    "DAY_OF_WEEK",
    "CRS_DEP_TIME",
    "OP_UNIQUE_CARRIER",
    "ORIGIN",
    "DEST",
    "DISTANCE"
]

df = df[columns]
df = df.sample(n=200000, random_state=42)

print("Sampled shape:", df.shape)


In [ ]:

df = df[(df["CANCELLED"] == 0) & (df["DIVERTED"] == 0)]
df = df.dropna(subset=["ARR_DELAY"])

df["DELAYED"] = np.where(df["ARR_DELAY"] > 15, 1, 0)

df = df.drop(columns=["ARR_DELAY", "DEP_DELAY", "CANCELLED", "DIVERTED"])

df.head()


In [ ]:

df["DEP_HOUR"] = (df["CRS_DEP_TIME"] // 100).astype(int)
df = df.drop(columns=["CRS_DEP_TIME"])

df.head()


In [ ]:

X = df.drop("DELAYED", axis=1)
y = df["DELAYED"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)


In [ ]:

categorical_features = ["OP_UNIQUE_CARRIER", "ORIGIN", "DEST"]
numeric_features = ["MONTH", "DAY_OF_WEEK", "DISTANCE", "DEP_HOUR"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
)


In [ ]:

log_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000))
])

log_model.fit(X_train, y_train)

y_pred_log = log_model.predict(X_test)
y_prob_log = log_model.predict_proba(X_test)[:,1]

print("Logistic Regression Results")
print(classification_report(y_test, y_pred_log))
print("ROC AUC:", roc_auc_score(y_test, y_prob_log))


In [ ]:

svm_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", SVC(kernel="rbf", probability=True))
])

svm_model.fit(X_train, y_train)

y_pred_svm = svm_model.predict(X_test)
y_prob_svm = svm_model.predict_proba(X_test)[:,1]

print("SVM Results")
print(classification_report(y_test, y_pred_svm))
print("ROC AUC:", roc_auc_score(y_test, y_prob_svm))


In [ ]:

rf_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(
        n_estimators=100,
        random_state=42,
        n_jobs=-1
    ))
])

rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
y_prob_rf = rf_model.predict_proba(X_test)[:,1]

print("Random Forest Results")
print(classification_report(y_test, y_pred_rf))
print("ROC AUC:", roc_auc_score(y_test, y_prob_rf))


In [ ]:

plt.figure(figsize=(8,6))

fpr_log, tpr_log, _ = roc_curve(y_test, y_prob_log)
fpr_svm, tpr_svm, _ = roc_curve(y_test, y_prob_svm)
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_prob_rf)

plt.plot(fpr_log, tpr_log, label="Logistic Regression")
plt.plot(fpr_svm, tpr_svm, label="SVM")
plt.plot(fpr_rf, tpr_rf, label="Random Forest")

plt.plot([0,1],[0,1],"k--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Comparison")
plt.legend()
plt.show()
